In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
import random
import glob

In [2]:
# # TRAIN_FILES = '/home/dataset/training*/*/data_smooth.csv'
# # TEST_FILES = '/home/dataset/validation*/*/data_smooth.csv'
# BOX_PTS = 9
# TRAIN_FILES = '/home/dataset/team2/training/training*/*_smooth_{}.csv'.format(BOX_PTS)
# TEST_FILES = '/home/dataset/team2/validation/validation*/*_smooth_{}.csv'.format(BOX_PTS)
    

# train_csvs = glob.glob(TRAIN_FILES)

# val_ratio = 0.1
# train_csv_num = len(train_csvs)

# val_num = int(train_csv_num * val_ratio)
# train_num = train_csv_num - val_num

# random.shuffle(train_csvs)

# split_train_csvs = train_csvs[:train_num]
# val_csvs = train_csvs[train_num:]
# train_csvs = split_train_csvs

# # print(train_csvs)
# # print(val_csvs)
# print('train csv num:', len(train_csvs))
# print('val csv num:', len(val_csvs))

# test_csvs = glob.glob(TEST_FILES)
# test_num = len(test_csvs)
# # print(test_csvs)
# print('test csv num:', len(test_csvs))

train csv num: 706
val csv num: 78


In [28]:
# use only car following
train_csvs = []
test_csvs = []
BOX_PTS = 9

file_list_path = '/home/dataset/urban_no_lane_changing_car_following.csv'
fl = pd.read_csv(file_list_path, header=None).values
for f in fl:
    file_name = f[0]
    if 'validation' in file_name:
        test_csvs.append(f[0] + '_smooth_{}.csv'.format(BOX_PTS))
    else:
        train_csvs.append(f[0] + '_smooth_{}.csv'.format(BOX_PTS))
        
file_list_path = '/home/dataset/highway_no_lane_changing_car_following.csv'
fl = pd.read_csv(file_list_path, header=None).values
for f in fl:
    file_name = f[0]
    if 'validation' in file_name:
        test_csvs.append(f[0] + '_smooth_{}.csv'.format(BOX_PTS))
    else:
        train_csvs.append(f[0] + '_smooth_{}.csv'.format(BOX_PTS))

val_ratio = 0.1
train_csv_num = len(train_csvs)

val_num = int(train_csv_num * val_ratio)
train_num = train_csv_num - val_num

random.shuffle(train_csvs)

split_train_csvs = train_csvs[:train_num]
val_csvs = train_csvs[train_num:]
train_csvs = split_train_csvs


print('train csv num:', len(train_csvs))
print('val csv num:', len(val_csvs))

test_num = len(test_csvs)
print('test csv num:', len(test_csvs))

train csv num: 80
val csv num: 8
test csv num: 39


In [29]:
max_frame = 200
dim_input = 12

output_dim = max_frame * 3

batch_size = 64
epochs = 300
n_hidden = 256

model_name = 'triple_lstm_epoch_{}_hidden_{}_car_following'.format(epochs, n_hidden)

def build_model():
    model = tf.keras.Sequential()
    model.add(layers.LSTM(n_hidden,return_sequences=True, dropout=0.25,recurrent_dropout=0.1,input_shape=(max_frame ,dim_input)))
    model.add(layers.LSTM(n_hidden, return_sequences=True, dropout=0.25,recurrent_dropout=0.1))
    model.add(layers.LSTM(n_hidden, return_sequences=False, dropout=0.25,recurrent_dropout=0.1))
    model.add(layers.Dense(output_dim))
#     adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    print(model.summary())
    return model

In [32]:
train_data = []
for file in train_csvs:
    try:
        raw = pd.read_csv(file).values
        length = raw.shape[0]
        padding = np.zeros((max_frame, dim_input + 3))
        padding[:length, :] = raw
        train_data.append(padding)
    except:
        print('file not exists: ',file)
    
train_data = np.array(train_data)

train_X = train_data[:,:,:-3]
train_y = train_data[:,:,-3:]
train_y = train_y.reshape(-1, output_dim)


test_data = []
for file in test_csvs:
    try:
        raw = pd.read_csv(file).values
        length = raw.shape[0]
        padding = np.zeros((max_frame, dim_input + 3))
        padding[:length, :] = raw
        test_data.append(padding)
    except:
        print('file not exists: ', file)

test_data = np.array(test_data)
test_X = test_data[:, :, :-3]
test_y = test_data[:, :, -3:]
test_y = test_y.reshape(-1, output_dim)

val_data = []
for file in val_csvs:
    try:
        raw = pd.read_csv(file).values
        length = raw.shape[0]
        padding = np.zeros((max_frame, dim_input + 3))
        padding[:length, :] = raw
        val_data.append(padding)
    except:
        print('file not exists: ', file)

val_data = np.array(val_data)
val_X = val_data[:, :, :-3]
val_y = val_data[:, :, -3:]
val_y = val_y.reshape(-1, output_dim)


print("train X: ", train_X.shape)
print("train y: ", train_y.shape)

print("test X: ", test_X.shape)
print("test y: ", test_y.shape)

print("val X: ", val_X.shape)
print("val y: ", val_y.shape)

file not exists:  /home/dataset/team2/training/training_0025/training_0025_segment-5835049423600303130_180_000_200_000_smooth_9.csv
file not exists:  /home/dataset/team2/training/training_0025/training_0025_segment-574762194520856849_1660_000_1680_000_smooth_9.csv
file not exists:  /home/dataset/team2/training/training_0025/training_0025_segment-5871373218498789285_3360_000_3380_000_smooth_9.csv
file not exists:  /home/dataset/team2/training/training_0025/training_0025_segment-6104545334635651714_2780_000_2800_000_smooth_9.csv
file not exists:  /home/dataset/team2/validation/validation_0002/validation_0002_segment-13336883034283882790_7100_000_7120_000_smooth_9.csv
file not exists:  /home/dataset/team2/validation/validation_0002/validation_0002_segment-13415985003725220451_6163_000_6183_000_smooth_9.csv
file not exists:  /home/dataset/team2/validation/validation_0003/validation_0003_segment-16979882728032305374_2719_000_2739_000_smooth_9.csv
file not exists:  /home/dataset/team2/valida

In [ ]:
# Aggregated Training Error
model = build_model()

history = model.fit(train_X, train_y, validation_data=(val_X, val_y), batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 256)          275456    
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 256)          525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 600)               154200    
Total params: 1,480,280
Trainable params: 1,480,280
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 76 samples, validate on 8 samples
Epoch 1/300
76/76 [==

In [ ]:
# Plot training & validation loss values
fig = plt.gcf()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
fig.savefig('/home/zg2309/history/{}'.format(model_name))
plt.close(fig)

score = model.evaluate(test_X, test_y, batch_size=batch_size)

In [ ]:
model.save("/home/zg2309/model/{}.h5".format(model_name))
print("Saved model to disk")